## Learning rate 
- Learning rate 를 처음에는 높게 하다가 나중에는 낮게 하는것
- 팁
    - **학습을 하다가 처음에는 0.1로 하다가 learning curve를 보다보니 특정 epochs 에서 좀 학습이 안된다 그러면 기억해 뒀다가 0.01 또 학습하다가 학습이 잘 안된다 그러면 0.001 로 이런식으로 하기**

In [1]:
import os
from glob import glob

from datetime import datetime

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import datasets

import matplotlib.pyplot as plt

%matplotlib inline

## Hyper parameter Tunning

In [2]:
num_epochs = 20
batch_size = 32
learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [15]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3,3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3,3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2,2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3,3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3,3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2,2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs = inputs, outputs = net)
model.summary()

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate),
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_12 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_13 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 16, 16, 32)        0   

## Data Process

In [4]:
## (1) train, test 데이터 읽어오기
print(os.listdir('../dataset/cifar/'))

train_paths = glob('../dataset/cifar/train/*.png')[:1000]
test_paths = glob('../dataset/cifar/test/*.png')[:1000]

print(len(train_paths), len(test_paths))

## (2) label name 반환
def get_class_name(path):
    return path.split('_')[-1].replace('.png','')

## (3) label name 고유값 list - class_names
train_labels = [get_class_name(path) for path in train_paths] # 50000개
class_names = np.unique(train_labels) # 해당 class 개로 줄여짐

# (4) label 반환 onehot encoding 된 형태로 or argmax 로 해당 index
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot) 

# (5) 최종 image, label 반환
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32)/.255 # rescale
    
    label = get_label(path)
    return image, label

# (6) image preprocess (랜덤 Generator)
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE # 작동하는 Network가 스스로 설정하고 Dataset을 잘 불러올 수 있게 결정하라는 것


# train dataset 뽑기
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls = AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls = AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size = len(train_paths))
train_dataset = train_dataset.repeat()

# test set 데이터 뽑기
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()
      

['labels.txt', 'test', 'train']
1000 1000


## Learning Rate Scheduler

In [12]:
import math

def scheduler(epoch):
    if epoch<10:
        return 0.001
    else:
        return 0.001 * math.exp(0.1 * (10-epoch))
    
for epoch in range(10, 20):
    print(scheduler(epoch))


0.001
0.0009048374180359595
0.0008187307530779819
0.0007408182206817179
0.0006703200460356394
0.0006065306597126335
0.0005488116360940264
0.0004965853037914095
0.0004493289641172216
0.00040656965974059914


In [13]:
learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)
print(learning_rate_scheduler)

## Training

In [16]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch = steps_per_epoch,
    validation_steps = validation_steps,
    epochs = num_epochs,
    callbacks = [learning_rate_scheduler]
)

Epoch 1/20
31/31 [==============================] - 4s 120ms/step - loss: 51.4288 - accuracy: 0.1126 - lr: 0.0010
Epoch 2/20
31/31 [==============================] - 6s 209ms/step - loss: 2.3746 - accuracy: 0.1129 - lr: 0.0010
Epoch 3/20
31/31 [==============================] - 7s 236ms/step - loss: 2.3602 - accuracy: 0.0900 - lr: 0.0010
Epoch 4/20
31/31 [==============================] - 8s 250ms/step - loss: 2.3327 - accuracy: 0.0950 - lr: 0.0010
Epoch 5/20
31/31 [==============================] - 7s 235ms/step - loss: 2.3179 - accuracy: 0.1343 - lr: 0.0010
Epoch 6/20
31/31 [==============================] - 8s 267ms/step - loss: 2.3090 - accuracy: 0.1169 - lr: 0.0010
Epoch 7/20
31/31 [==============================] - 6s 202ms/step - loss: 2.3165 - accuracy: 0.0985 - lr: 0.0010
Epoch 8/20
31/31 [==============================] - 6s 183ms/step - loss: 2.3172 - accuracy: 0.1064 - lr: 0.0010
Epoch 9/20
31/31 [==============================] - 5s 174ms/step - loss: 2.3096 - accuracy: 0.